In [1]:
from bottle import Bottle, route, run
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pprint


url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest'
parameters = {
  'slug':'bitcoin',
  'convert':'PLN'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '0338f88c-4dc5-4997-b9dd-a7dd973e4a0d',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  pprint.pprint(json.loads(response.text)['data']['1']['quote']['USD']['price'])
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

app = Bottle()

@app.route('/index')
def hello():
    return '<h1>Hello</h1>'

run(app, host = 'localhost', port = 8080, debug = True)

48819.75913664301


Bottle v0.12.19 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.

C:\Users\leski\anaconda3\lib\site-packages\bottle.py:3139: ResourceWarning: unclosed <socket.socket fd=1348, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 8080)>
  pass


In [45]:
#commit Jarek 22.12
#Poprawilem api, trzeba po innym linku sie laczyc aby seryjnie zaciagac 

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pprint

import pandas as pd

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'

currency = 'USD'

parameters = {
  'convert': currency
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '25bdeaa9-d947-4b43-83b5-f2ee7bb97197',
}

session = Session()
session.headers.update(headers)
coins_names = []
coins_prices = []
coins_volume_24h = []
coins_market_cap = []

try:
    response = session.get(url, params=parameters)
    
except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)
    
for coins in range(100):
    coins_names.append((json.loads(response.text)['data'][coins]['name']))
    coins_prices.append((json.loads(response.text)['data'][coins]['quote'][currency]['price']))
    coins_volume_24h.append((json.loads(response.text)['data'][coins]['quote'][currency]['volume_24h']))
    coins_market_cap.append((json.loads(response.text)['data'][coins]['quote'][currency]['market_cap']))
                          
rtime = json.loads(response.text)['data'][1]["last_updated"]
data = list(zip(coins_names,coins_prices, coins_volume_24h, coins_market_cap))
dataframe = pd.DataFrame(data, columns = ['Crypto', 'Price', 'volume 24h', 'Market Cap'])

print("Update time:", rtime)
print(dataframe)



<ipython-input-45-c12c99e9d58b>:23: ResourceWarning: unclosed <ssl.SSLSocket fd=400, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.8.132', 59636), raddr=('104.17.141.178', 443)>
  response = session.get(url, params=parameters)


Update time: 2021-12-22T19:36:02.000Z
          Crypto         Price    volume 24h    Market Cap
0        Bitcoin  48806.116088  2.454316e+10  9.228263e+11
1       Ethereum   3988.321817  1.418698e+10  4.740007e+11
2   Binance Coin    538.378190  1.544596e+09  8.980210e+10
3         Tether      1.000456  5.777829e+10  7.625437e+10
4         Solana    180.494575  1.884445e+09  5.567722e+10
..           ...           ...           ...           ...
95   OMG Network      6.631274  3.322394e+08  9.300056e+08
96          BORA      1.064338  1.315970e+08  9.166611e+08
97     Ravencoin      0.087859  3.210815e+07  9.100392e+08
98        Revain      0.010251  2.000627e+06  8.720078e+08
99          Ankr      0.106035  5.860936e+07  8.655504e+08

[100 rows x 4 columns]


In [39]:
type(coins_names)

list